In [2]:
import os
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
from dash import dash_table
from dash.dash_table.Format import Group
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc
from navbar_tabs_layout import app_layout
import time
from datetime import datetime, date, time, timedelta
from dateutil.relativedelta import relativedelta
import mysql.connector
from flask_caching import Cache

In [3]:
# Read the data
# df = pd.read_csv('EZItotalFull.csv')

In [4]:
# Define a function to fetch data from the database
def fetch_data():
    # Connect to the database
    conn = mysql.connector.connect(
        user = 'readonly',
        password = 'MwLvD9DFL8mnrTKkI6fU', 
        host = '127.0.0.1',
        #charset='utf8mb4',
        port=int(3335))
        #database=database,
        #port=int(3306))
    
    # Fetch data from SQL queries
    query = """
        SELECT i.sales_order_id, i.id, i.quantity, i.product_id, i.state, i.status, i.created_at, i.external_unit_price,
       p.id AS sku_id, p.sku, p.name, p.description, p.currency, p.brand_id, p.face_price, p.country, b.name,
       o.date, o.invoice_number, o.state, o.customer_company_id, c.id AS cust_id, c.name, a.city, a.country, x.finance_number, x.sales_order_id
        FROM (SELECT *
              FROM ezscm_production.sales_order_items
              WHERE created_at BETWEEN '2019-01-01' AND '2023-12-31') AS i
        LEFT OUTER JOIN (SELECT *
                         FROM ezscm_production.sales_orders
                         WHERE date BETWEEN '2019-01-01' AND '2023-12-31') AS o
            ON o.id = i.sales_order_id
        LEFT OUTER JOIN ezscm_production.products AS p
            ON p.id = i.product_id
        LEFT OUTER JOIN ezscm_production.brands AS b
            ON p.brand_id = b.id
        LEFT OUTER JOIN ezscm_production.companies AS c
            ON o.customer_company_id = c.id
        LEFT OUTER JOIN ezscm_production.addresses AS a
            ON c.billing_address_id = a.id
        LEFT OUTER JOIN ezscm_production.invoices AS x
            ON o.invoice_number = x.finance_number

        """
    df = pd.read_sql(query,conn)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)

    # Close the database connection
    conn.close()
    
    return df

In [5]:
#df = fetch_data()

In [7]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.UNITED], meta_tags=[{"name": "viewport", "content": "width=device-width"}])
app.title = 'Dashboard'

In [8]:
cache = Cache(app.server, config={
    'CACHE_TYPE': 'filesystem',
    'CACHE_DIR': 'cache-directory'
})

In [9]:
today_date = datetime.now().strftime("%Y-%m-%d")
# df['date'] = pd.to_datetime(df['date'])
# df['year'] = df['date'].dt.year
# df['month'] = df['date'].dt.month

In [10]:
navbar = dbc.Navbar(
    dbc.Container(
        [
            dbc.Row(
                [
                    dbc.Col(html.Img(src="db.png", height="20px"), width="auto", align="center"),
                    dbc.Col(dbc.NavbarBrand("My Dashboard", className="ml-2"), width="auto", align="center"),
                    dbc.Col(html.Div(id="today-date", children=today_date), width="auto", align="center"),
                    dbc.Col(
                        [
                            dcc.Loading(
                                id="loading-fetch-data",
                                children=[
                                    dbc.Button('Fetch Data', id='fetch-data-button', className="mr-2")
                                ],
                                type="circle",
                            ),
                            dbc.Button(id='query-status-button', disabled=True),
                        ],
                        width="auto", align="end"),
                ],
                className="my-row",
                align="center",
            ),
            dbc.NavbarToggler(id="navbar-toggler"),
        ]
    ),
    color="light",
    dark=False,
    sticky="top",
)

In [11]:
# # Define the data-table and store components outside the navbar
# data_table = dcc.Loading(
#     dash_table.DataTable(
#         id='data-table',
#         style_data={
#             'whiteSpace': 'normal',
#             'height': 'auto'
#         },
#         style_cell={
#             'textAlign': 'left',
#             'font_family': 'Open Sans'
#         }
#     )
# )
# data_store = dcc.Store(id='csv-filename-store', data='data.csv')


In [12]:
# Define callback to fetch data
@app.callback(
    [
        Output("data-store", "data"),
        Output("query-status-button", "disabled"),
        Output("loading-fetch-data", "fullscreen"),
    ],
    [Input("fetch-data-button", "n_clicks")],
    [State("data-store", "data")]
)
def fetch_data_callback(n_clicks, data):
    # If the button has not been clicked yet, return the input data as is
    if n_clicks is None:
        return dash.no_update, dash.no_update, False

    # Display loading screen while the data is being fetched
    time.sleep(2)
    with open("data.json") as f:
        df = pd.read_json(f.read(), orient="split")

    # Store the fetched data in session
    session["df"] = df.to_json(date_format='iso', orient='split')

    return session["df"], False, False

In [18]:
data_store = dcc.Store(id='data-store')

In [14]:
# show_data_button = html.Button('Show Data', id='show-data-button')

In [15]:
# @app.callback(Output('data-store', 'data'),
#               Input('fetch-data-button', 'n_clicks'))
# def update_data_store(n_clicks):
#     if n_clicks:
#         df = fetch_data()
#         return df.to_dict('records')


In [16]:
# @app.callback(
#     Output('output-div', 'children'),
#     [Input('fetch-data-button', 'n_clicks'),
#      Input('show-data-button', 'n_clicks')],
#     State('data-store', 'data')
# )
# def update_output_div(fetch_clicks, show_clicks, data):
#     if not any([fetch_clicks, show_clicks]):
#         raise dash.exceptions.PreventUpdate

#     ctx = dash.callback_context
#     if ctx.triggered[0]['prop_id'] == 'fetch-data-button.n_clicks':
#         return 'Fetching data...'

#     df = pd.read_csv(data)
#     table = dcc.Loading(
#         dash_table.DataTable(
#             id='data-table',
#             data=df.to_dict('records'),
#             columns=[{'name': i, 'id': i} for i in df.columns],
#             style_data={
#                 'whiteSpace': 'normal',
#                 'height': 'auto'
#             },
#             style_cell={
#                 'textAlign': 'left',
#                 'font_family': 'Open Sans'
#             }
#         )
#     )
#     return table

In [23]:
app.layout = dbc.Container(
    [
        navbar,
        data_store,
        dbc.Row(
            dbc.Col(
                dcc.Loading(
                    id='loading-fetch-data',
                    children=[
                        dbc.Button("Fetch Data", id='fetch-data-button', color='primary', className='mt-3'),
                        html.Div(id='fetch-data-output')
                    ],
                    type='circle',
                )
            )
        ),
        dbc.Row(
            dbc.Col(
                dcc.Loading(
                    id='loading-status',
                    children=[
                        html.Div(id='status-div')
                    ],
                    type='circle',
                )
            )
        ),
        dbc.Row(
            dbc.Col(
                dbc.Button("Show Data", id="show-data-button", color="primary", className="mt-3")
            )
        ),
    ],
    fluid=True
)


In [24]:
# Set the app layout
#app.layout = app_layout
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on


In [ ]:
# Define page layout
page_layout = dbc.Container([
    
    # Add top_level section
    dbc.Row([
                dbc.Col(dcc.DatePickerSingle(
                    id='date-picker',
                    min_date_allowed=df['date'].min(),
                    max_date_allowed=df['date'].max(),
                    initial_visible_month=df['date'].max(),
                    date=df['date'].max()
                ),
                width=3
        ),
        dbc.Col(dbc.Card([
                    dbc.CardHeader("Overnight Orders, including not Complete"),
                    dbc.CardBody([html.P(id='overnight-sales')])
                ]),
                width=8
        ),

    ], justify="between", align="center", className='mb-4'),

    dbc.Card([
        dbc.CardBody([
            dbc.Row([
                dbc.Col([
                    dbc.Card([
                        dbc.CardHeader("Sales YtD"),
                        dbc.CardBody([
                            dcc.Graph(id='sales-chart-2')
                        ])
                    ], className="h-100", style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'})
                ], sm=12, lg=4, className="mb-4"),
                dbc.Col([
                    dbc.Card([
                        dbc.CardHeader("Sales Mtd"),
                        dbc.CardBody([
                            dcc.Graph(id='sales-chart')
                        ])
                    ], className="h-100", style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'})
                ], sm=12, lg=4, className="mb-4"),
                dbc.Col([
                    dbc.Card([
                        dbc.CardHeader(
                            [html.H6("Customers to chase"), html.Button('Render Table', id='render-button', className='btn-sm')],
                            style={'backgroundColor': '#F8F9FA'}
                        ),
                        dbc.CardBody(
                            dcc.Graph(
                                id='table',
                                config={'displayModeBar': False}
                            ),
                            style={'padding': '0', 'margin': '0'}
                        )
                    ], className="h-100", style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'})
                ], sm=12, lg=4, className="mb-4")
            ])
        ])
    ], className="rounded-0 border-0"),

    # Add rest of page layout here
    html.Div("Click on below tabs for further analysis 👇"),

], fluid=True, style={'padding': '2rem'})


In [66]:
# #displaying gauge instead of bar chart

def new_sales_chart(date, df):
    current_year = pd.Timestamp(date).year
    current_month = pd.Timestamp(date).month
    current_day = pd.Timestamp(date).day
    prev_year = current_year - 1
    two_year = current_year - 2
    
    ytd_start = pd.to_datetime(f"{current_year}-01-01")
    last_ytd_start = pd.to_datetime(f"{prev_year}-01-01")
    last_ytd_end = pd.to_datetime(f"{prev_year}-{current_month}-{current_day}")
    two_ytd_start = pd.to_datetime(f"{two_year}-01-01")
    two_ytd_end = pd.to_datetime(f"{two_year}-{current_month}-{current_day}")

    sales_ytd = round(df[(df['date'] >= ytd_start)]['total'].sum(),2)
    sales_ytd_formatted = "{:,.2f}".format(sales_ytd)
    sales_ytd_last_year = df[(df['date'] >=last_ytd_start)& (df['date'] <= last_ytd_end)]['total'].sum()
    sales_ytd_two_year = df[(df['date'] >=two_ytd_start)& (df['date'] <= two_ytd_end)]['total'].sum()
    percent_vs_last_year_to_date = round(sales_ytd/sales_ytd_last_year*100,2)
    percent_vs_two_year_to_date = sales_ytd/sales_ytd_two_year 

    fig = go.Figure(go.Indicator(
        mode="gauge+number",
        value=percent_vs_last_year_to_date,
        gauge={
            'axis': {'range': [None, 100], 'tickwidth': 1, 'tickcolor': 'darkblue'},
            'bar': {'color': 'deepskyblue'},
            'bgcolor': 'white',
            'borderwidth': 2,
            'bordercolor': 'gray',
            'steps': [{'range': [0, 50], 'color': 'white'},
                      {'range': [50, 100], 'color': 'white'},
                      {'range': [100, 150], 'color': 'white'},
                      {'range': [1500, 200], 'color': 'white'}]
        },
        domain={'x': [0, 1], 'y': [0, 1]},
        #title={'text': "% of YtD vs past YtD sales"}
    ))

    # Add the sales_ytd string as a markdown string to the layout of the fig variable
    fig.update_layout(
        annotations=[
            go.layout.Annotation(
                x=0.5,
                y=1.2,
                showarrow=False,
                text=f"YTD Sales: $ {sales_ytd_formatted}",
                xref="paper",
                yref="paper",
                align='center',
                font=dict(size=18)
            ),
            go.layout.Annotation(
                x=0.5,
                y=-0.2,
                showarrow=False,
                text=f"{percent_vs_last_year_to_date}% generated vs past YtD sales",
                xref="paper",
                yref="paper",
                align='center',
                font=dict(size=18)
            )
        ]
    )
    
    return fig
#new_sales_chart(today_date, df)

In [67]:
new_sales_chart(today_date, df)

In [ ]:
#displaying a gauge instead of bar chart 
@app.callback(
    Output('sales-chart-2', 'figure'),
    [Input('date-picker', 'date')])
def updated_new_sales_chart(date):
    # Call the new_sales_chart function to generate the figure
    fig = new_sales_chart(date, df)
    
    return fig

In [ ]:
def create_sales_chart(df, date):
    month = pd.Timestamp(date).to_pydatetime()
    sales_mtd = df[(df['month'] == month.month) & (df['year'] == month.year)]['total'].sum()
    sales_mtd_formatted = "{:,.2f}".format(sales_mtd)
    sales_mtd_last_year = df[(df['month'] == month.month) & (df['year'] == month.year - 1)]['total'].sum()
    sales_mtd_two_year = df[(df['month'] == month.month) & (df['year'] == month.year - 2)]['total'].sum()
    percent_vs_last_year_month = round(sales_mtd/sales_mtd_last_year*100,2)
    percent_vs_two_year_month = round(sales_mtd/sales_mtd_two_year*100,2)

    labels = ['vs 2Y ago', 'vs last year','MtD']
    category_names = [sales_mtd_two_year, sales_mtd_last_year, sales_mtd_formatted]

    trace = go.Bar(
        x=category_names,
        y=labels,
        orientation='h',
        text=category_names,
        textposition='auto',
        marker=dict(
            color='deepskyblue'
        )
    )

    data = [trace]
    layout = go.Layout(
        #xaxis=dict(title='Total Sales'),
        #yaxis=dict(title='Category'),
        margin=dict(l=1)
    )

    fig = go.Figure(data=data, layout=layout)
    return fig
#create_sales_chart(df, today_date)

In [ ]:
@app.callback(
    Output('sales-chart', 'figure'),
    [Input('date-picker', 'date')]
)
def update_sales_chart(date):
    month = pd.Timestamp(date).to_pydatetime()
    sales_chart = create_sales_chart(df, month)
    return sales_chart


In [ ]:
today = pd.Timestamp.now().floor('D')
def prev_weekday(adate):
    adate -= timedelta(days=1)
    while adate.weekday() > 4: # Mon-Fri are 0-4
        adate -= timedelta(days=1)
    return adate
overnight_sales = df[(df['date'] >= prev_weekday(today))]['total'].sum()


In [ ]:
@app.callback(
    Output('overnight-sales', 'children'),
    [Input('date-picker', 'date')]
)
def update_overnight_sales(date):
    # Calculate overnight sales based on selected date
    overnight_sales = round(df[(df['date'] >= pd.Timestamp(date).floor('D'))]['total'].sum(),2)
    overnight_sales = "{:,.2f}".format(overnight_sales)
    return f"Overnight sales = {overnight_sales}"
#update_overnight_sales(today_date)

In [ ]:
def get_customers_to_chase(df):
    # Filter for customers who last ordered this year
    current_year = datetime.now().year
    
    # How long have we got each cust
    customer_length = df.groupby('name_o').agg({'sales_order_id_o': 'nunique', 'date': ['min', 'max']})
    customer_length['longevity'] = customer_length['date']['max'] - customer_length['date']['min']
    customer_length = customer_length.reset_index()
    customer_length.columns = ['name_o', 'orders_count', 'first_order', 'last_order', 'longevity']
    customer_length['last_order'] = pd.to_datetime(customer_length['last_order'], format='%Y-%m-%d')
    filtered_df = customer_length.loc[(customer_length['last_order'].dt.year == current_year)]
    sorted_df = filtered_df[['name_o', 'orders_count', 'last_order', 'longevity']].sort_values('last_order').head(10)
    # Format the longevity column in years and months
    sorted_df['longevity'] = sorted_df['longevity'].apply(lambda x: f'{int(x.days/365)}y {int((x.days%365)/30)}m')
    sorted_df['last_order'] = pd.to_datetime(sorted_df['last_order'], format='%Y-%m-%d').dt.strftime('%d-%m-%Y')

    fig = go.Figure(data=[go.Table(
        columnwidth=1,
        header=dict(values=list(sorted_df.columns),
                    fill_color='lightgray',
                    align=['left', 'center']),
        cells=dict(values=[sorted_df.name_o, sorted_df.orders_count, sorted_df.last_order, sorted_df.longevity],
                   fill_color='white',
                   align=['left', 'center']))
    ])

    return fig

get_customers_to_chase(df)


In [ ]:
@app.callback(
    Output('table', 'figure'),
    Input('render-button', 'n_clicks'),
    #State('my-dataframe', 'data')
)
def update_table(n_clicks):
    if not n_clicks:
        raise PreventUpdate
    table = get_customers_to_chase(df)
    return table
update_table(1)

In [ ]:
# # Filter for customers who last ordered this year
# current_year = datetime.now().year
# #How long have we got each cust 
# customer_length = df.groupby('name_o').agg({'sales_order_id_o': 'nunique','date': ['min', 'max']})
# customer_length['longevity'] = customer_length['date']['max'] - customer_length['date']['min']
# customer_length = customer_length.reset_index()
# customer_length.columns = ['name_o', 'orders_count', 'first_order', 'last_order','longevity']
# filtered_df = customer_length.loc[(customer_length['last_order'].dt.year == current_year)]
# filtered_df.sort_values('last_order').head(10)

In [ ]:
# Sales dashboard
sales_tab_content = dbc.Container([
     dbc.Row([
        dbc.Col(html.H6('...', className='card-title mb-4')),
        html.Label('Select year:'), 
        dcc.Dropdown(
                            id='year-dropdown',
                            options=[{'label': str(year), 'value': year} for year in df['year'].unique()],
                            value=df['year'].min()
                        ),
        ], justify="between", align="center", className='mb-4'),

    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("Sales by Top 3 Customers")),
                dbc.CardBody(
                    [
                        dcc.Graph(id='sales-graph-1')
                    ], 
                    style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
                ),
            ], 
            style={'height': '100%'}
            )
        ], md=6),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("Sales vs Order vs Total SKU")),
                dbc.CardBody(
                    [
                        dcc.Graph(id='sales-graph-2')
                    ],
                    style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
                ),
            ], 
            style={'height': '100%'}
            )
        ], md=6),
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("Average Order Value")),
                dbc.CardBody(
                    [
                        dcc.Graph(id='sales-graph-3')
                    ],
                    style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
                ),
            ], 
            style={'height': '100%'}
            )
        ], md=6),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("Sales by Region")),
                dbc.CardBody(
                    [
                        dcc.Graph(id='sales-graph-4')
                    ],
                    style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
                ),
            ], 
            style={'height': '100%'}
            )
        ], md=6),
    ]),
], fluid=True, style={'padding': '2rem'})


In [ ]:
# Calculate total sales by customer over the entire period
total_sales_by_customer = df.groupby('name_o')['total'].sum()

# Select top 20 customers based on their total sales
top_customers = total_sales_by_customer.nlargest(20).index.tolist()

# Create a dictionary of colors for the top 20 customers
customer_colors = {customer: px.colors.qualitative.Pastel1[i % len(px.colors.qualitative.Pastel1)]
                   for i, customer in enumerate(top_customers)}

# Add a color for 'others'
customer_colors['others'] = 'gray'

# Assign colors to each customer using the color mapping dictionary
def assign_color(customer):
    if customer in customer_colors:
        return customer_colors[customer]
    else:
        return 'gray'

@app.callback(
    Output('sales-graph-1', 'figure'),
    [Input('year-dropdown', 'value')])
def update_sales_graph_1(selected_year):
    filtered_df = df[df['year'] == selected_year]

    # Calculate total sales by month and customer
    sales_by_month_customer = filtered_df.groupby(['month', 'name_o'])['total'].sum().reset_index()

    # Find top customers for each month
    top_customers_by_month = sales_by_month_customer.groupby('month').apply(lambda x: x.nlargest(3, 'total')).reset_index(drop=True)

    # Group the remaining customers as "others"
    remaining_customers_by_month = sales_by_month_customer[~sales_by_month_customer['name_o'].isin(top_customers)]
    remaining_customers_by_month = remaining_customers_by_month.groupby('month').sum().reset_index()
    remaining_customers_by_month['name_o'] = 'others'

    # Concatenate the top customers and the "others" dataframes
    sales_by_month_customer = pd.concat([top_customers_by_month, remaining_customers_by_month])

    # Assign colors to each customer using the color mapping dictionary
    sales_by_month_customer['color'] = sales_by_month_customer['name_o'].apply(assign_color)

    # Create stacked bar chart of sales by customer
    fig = px.bar(sales_by_month_customer, x='month', y='total', color='name_o', color_discrete_map=customer_colors)
    fig.update_layout(
        title=f'Top 3 customers and others by month ({selected_year})',
        xaxis_title='Month',
        yaxis_title='Total Sales ($)',
        barmode='stack'
    )
    return fig
#update_sales_graph_1(2021)

In [ ]:
@app.callback(
    Output('sales-graph-2', 'figure'),
    [Input('year-dropdown', 'value')])
def update_sales_graph_2(selected_year):
    filtered_df = df[df['year'] == selected_year]
    
    # Total Sales trend by month
    sales_by_month = filtered_df.groupby(pd.Grouper(key='date', freq='M')).sum()['total']
    sales_trace = go.Scatter(x=sales_by_month.index, y=sales_by_month.values, mode='lines', name='Total Sales')

    # Total Unique Orders trend by month
    unique_orders_by_month = filtered_df.groupby(pd.Grouper(key='date', freq='M'))['id_o'].nunique()
    unique_orders_trace = go.Scatter(x=unique_orders_by_month.index, y=unique_orders_by_month.values, mode='lines', name='Total Unique Orders')

    # Total SKU trend by month
    total_sku_by_month = filtered_df.groupby(pd.Grouper(key='date', freq='M'))['quantity'].sum()
    total_sku_trace = go.Scatter(x=total_sku_by_month.index, y=total_sku_by_month.values, mode='lines', name='Total SKUs')
    
    fig = go.Figure()
    
    fig.add_trace(sales_trace)
    fig.add_trace(unique_orders_trace)
    fig.add_trace(total_sku_trace)

    fig.update_layout(
        font=dict(size=10),
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="white",
        plot_bgcolor="white",
        xaxis_title='Month',
        yaxis_title='Total',
        title="Sales, Unique Orders, and SKUs Trend"
    )

    return fig
#update_sales_graph_2(2021)

In [ ]:
clients_tab_content = dbc.Card(
    dbc.CardBody("tab to fill per client")
)

In [ ]:
prospects_tab_content = dbc.Card(
    dbc.CardBody("habits and nudge")
)

In [ ]:
# Define tabs
tabs = dbc.Tabs(
    [
        dbc.Tab(sales_tab_content, label="Historic Sales"),
        dbc.Tab(clients_tab_content, label="Customer Analysis"),
        dbc.Tab(prospects_tab_content, label="Prospects Market Research"),
    ],
    id="tabs",
    active_tab="sales_tab_content",
)

In [98]:
# # Define app layout
# app.layout = html.Div([
#     navbar,
#     dcc.Store(id="df"),
#     page_layout,
#     dbc.Container([
#         dbc.Row([
#             dbc.Col([
#                 tabs,
#             ]),
#         ], className='mt-4')
#     ])
# ], style={'padding': '0rem 0rem 2rem 0rem'})

In [130]:
# # Update the page content based on the url
# @app.callback(Output("page-content", "children"), [Input("url", "pathname")])
# def render_page_content(pathname):
#     if pathname == "/":
#         return landing_page
#     elif pathname == "/apps/sales":
#         return sales_dashboard
#     elif pathname == "/apps/clients":
#         return clients_dashboard
#     elif pathname == "/apps/prospects":
#         return prospects_dashboard
#     else:
#         return html.P("Page not found")
